<a href="https://colab.research.google.com/github/mahainn/NLP-Sentiment-Analysis/blob/main/NLP_Updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tweet-preprocessor 2>/dev/null 1>/dev/null


In [ ]:
! sudo apt install openjdk-8-jdk
! sudo update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
! pip install language-check
! pip install pycontractions



Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libxxf86dga1 openjdk-8-jdk-headless openjdk-8-jre
  openjdk-8-jre-headless x11-utils
Suggested packages:
  gvfs openjdk-8-demo openjdk-8-source visualvm icedtea-8-plugin libnss-mdns
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libxxf86dga1 openjdk-8-jdk openjdk-8-jdk-headless
  openjdk-8-jre openjdk-8-jre-headless x11-utils
0 upgraded, 15 newly installed, 0 to remove and 14 not upgraded.
Need to get 43.4 MB of archives.
After this 

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("/content/drive/My Drive/NLP/text_emotion.csv")

In [ ]:
data.shape

(40000, 4)


In [ ]:
data.head(20)

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...
5,1956968477,worry,xxxPEACHESxxx,Re-pinging @ghostridah14: why didn't you go to...
6,1956968487,sadness,ShansBee,"I should be sleep, but im not! thinking about ..."
7,1956968636,worry,mcsleazy,Hmmm. http://www.djhero.com/ is down
8,1956969035,sadness,nic0lepaula,@charviray Charlene my love. I miss you
9,1956969172,sadness,Ingenue_Em,@kelcouch I'm sorry at least it's Friday?


In [ ]:
misspell_data = pd.read_csv("/content/drive/MyDrive/NLP/wikipedia.txt",sep=":",names=["correction","misspell"])
misspell_data.misspell = misspell_data.misspell.str.strip()
misspell_data.misspell = misspell_data.misspell.str.split(" ")
misspell_data = misspell_data.explode("misspell").reset_index(drop=True)
misspell_data.drop_duplicates("misspell",inplace=True)
miss_corr = dict(zip(misspell_data.misspell,misspell_data.correction ))

#Sample of the dict
{v:miss_corr[v] for v in [list(miss_corr.keys())[k] for k in range(20)]}

{'Apenines': 'Apennines',
 'Appenines': 'Apennines',
 'Athenean': 'Athenian',
 'Atheneans': 'Athenians',
 'Bernouilli': 'Bernoulli',
 'Blitzkreig': 'Blitzkrieg',
 'Brasillian': 'Brazilian',
 'Britian': 'Britain',
 'Brittish': 'British',
 'Cambrige': 'Cambridge',
 'Carribean': 'Caribbean',
 'Carthagian': 'Carthaginian',
 'Cataline': 'Catalina',
 'Ceasar': 'Caesar',
 'Celcius': 'Celsius',
 'Champange': 'Champagne',
 'Conneticut': 'Connecticut',
 'Cyprian': 'Cypriot',
 'carcas': 'Caracas',
 'eles': 'Ellis'}

In [ ]:
def misspelled_correction(val):
    for x in val.split(): 
        if x in miss_corr.keys(): 
            val = val.replace(x, miss_corr[x]) 
    return val

data["clean_content"] = data.content.apply(lambda x : misspelled_correction(x))

In [ ]:
data.head(20)

,tweet_id,sentiment,author,content,clean_content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...,@dannycastillo We want to trade with someone w...
5,1956968477,worry,xxxPEACHESxxx,Re-pinging @ghostridah14: why didn't you go to...,Re-pinging @ghostridah14: why didn't you go to...
6,1956968487,sadness,ShansBee,"I should be sleep, but im not! thinking about ...","I should be sleep, but im not! thinking about ..."
7,1956968636,worry,mcsleazy,Hmmm. http://www.djhero.com/ is down,Hmmm. http://www.djhero.com/ is down
8,1956969035,sadness,nic0lepaula,@charviray Charlene my love. I miss you,@charviray Charlene my love. I miss you
9,1956969172,sadness,Ingenue_Em,@kelcouch I'm sorry at least it's Friday?,@kelcouch I'm sorry at least it's Friday?


In [ ]:
#execute from here


In [ ]:
#contractions part without using a dataset

from pycontractions import Contractions
#from openerp import api,models,fields

#model = api.load("glove-twitter-25")
#cont = Contractions(kv_model=model)
#cont.load_models()
cont = Contractions(api_key="glove-twitter-25")
cont.load_models()

def expand_contractions(text):
    text = list(cont.expand_texts([text], precise=True))[0]
    return text



In [ ]:
data.clean_content = data.clean_content.apply(lambda x : expand_contractions(x))
#this might take some time too

In [ ]:
data.head(40)

,tweet_id,sentiment,author,content,clean_content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...,@dannycastillo We want to trade with someone w...
5,1956968477,worry,xxxPEACHESxxx,Re-pinging @ghostridah14: why didn't you go to...,Re-pinging @ghostridah14: why did not you go t...
6,1956968487,sadness,ShansBee,"I should be sleep, but im not! thinking about ...","I should be sleep, but I am not! thinking abou..."
7,1956968636,worry,mcsleazy,Hmmm. http://www.djhero.com/ is down,Hmmm. http://www.djhero.com/ is down
8,1956969035,sadness,nic0lepaula,@charviray Charlene my love. I miss you,@charviray Charlene my love. I miss you
9,1956969172,sadness,Ingenue_Em,@kelcouch I'm sorry at least it's Friday?,@kelcouch I am sorry at least it is Friday?


In [ ]:
# remove UrL and username tags/mentions
import preprocessor as p
p.set_options(p.OPT.MENTION, p.OPT.URL)
p.clean("Naturallanguageprocessing @alx #sentimentAnalysis🔥 1245 https://github.com/s/preprocessor")

'Naturallanguageprocessing #sentimentAnalysis🔥 1245'

In [ ]:
data.clean_content = data.clean_content.apply(lambda x : p.clean(x))


In [ ]:
data.head(20)

,tweet_id,sentiment,author,content,clean_content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...,i know i was listenin to bad habit earlier and...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...,Layin n bed with a headache ughhhh...waitin on...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...,We want to trade with someone who has Houston ...
5,1956968477,worry,xxxPEACHESxxx,Re-pinging @ghostridah14: why didn't you go to...,Re-pinging : why did not you go to prom? BC my...
6,1956968487,sadness,ShansBee,"I should be sleep, but im not! thinking about ...","I should be sleep, but I am not! thinking abou..."
7,1956968636,worry,mcsleazy,Hmmm. http://www.djhero.com/ is down,Hmmm. is down
8,1956969035,sadness,nic0lepaula,@charviray Charlene my love. I miss you,Charlene my love. I miss you
9,1956969172,sadness,Ingenue_Em,@kelcouch I'm sorry at least it's Friday?,I am sorry at least it is Friday?




#Removing Punctuations and Emojis

In [37]:
def punctuation(val): 
  
    punctuations = '''()-[]{};:'"\,<>.!?/@#$%^&_~'''
  
    for x in val.lower(): 
        if x in punctuations: 
            val = val.replace(x, " ") 
    return val

In [38]:
#example
punctuation("NLP @ #seNTIMENTaNALYSIS??? !! ")

'NLP    seNTIMENTaNALYSIS       '

In [39]:
!pip install emoji


     |████████████████████████████████| 51kB 3.9MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49716 sha256=737b07296a2520187f15e6d12569b3b3ad9734f81f5bf0da60886a627f37a9f5
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji


In [41]:
import emoji

data.clean_content = data.clean_content.apply(lambda x : ' '.join(punctuation(emoji.demojize(x)).split()))

In [42]:
data.head(20)

,tweet_id,sentiment,author,content,clean_content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...,i know i was listenin to bad habit earlier and...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...,Layin n bed with a headache ughhhh waitin on y...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...,Funeral ceremony gloomy friday
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!,wants to hang out with friends SOON
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...,We want to trade with someone who has Houston ...
5,1956968477,worry,xxxPEACHESxxx,Re-pinging @ghostridah14: why didn't you go to...,Re pinging why did not you go to prom BC my bf...
6,1956968487,sadness,ShansBee,"I should be sleep, but im not! thinking about ...",I should be sleep but I am not thinking about ...
7,1956968636,worry,mcsleazy,Hmmm. http://www.djhero.com/ is down,Hmmm is down
8,1956969035,sadness,nic0lepaula,@charviray Charlene my love. I miss you,Charlene my love I miss you
9,1956969172,sadness,Ingenue_Em,@kelcouch I'm sorry at least it's Friday?,I am sorry at least it is Friday


#Removing Stop words


In [43]:
def removeStopWords(stopWords, txt):
  newtxt = ' '.join([word for word in txt.split() if word not in stopWords])
  return newtxt

#sents_processed = [removeStopWords(stop_set,x) for x in sents_lower]



In [49]:
data.clean_content = data.clean_content.apply(lambda x : x.lower())


In [51]:
stop_set = ['the', 'a', 'an', 'i', 'he', 'she', 'they', 'to', 'of', 'from','me', 'my', 'myself', 'we', 'our', 'ours','yourself', 'yourselves', 'him', 'his', 'himself','her','hers', 'herself','them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that','am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', ]

data.clean_content = data.clean_content.apply(lambda x : removeStopWords(stop_set,x))


In [52]:
data.head(20)

,tweet_id,sentiment,author,content,clean_content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...,know listenin bad habit earlier and started fr...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...,layin n bed with headache ughhhh waitin on you...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...,funeral ceremony gloomy friday
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!,wants hang out with friends soon
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...,want trade with someone houston tickets but no...
5,1956968477,worry,xxxPEACHESxxx,Re-pinging @ghostridah14: why didn't you go to...,re pinging why not you go prom bc bf not like ...
6,1956968487,sadness,ShansBee,"I should be sleep, but im not! thinking about ...",should sleep but not thinking about old friend...
7,1956968636,worry,mcsleazy,Hmmm. http://www.djhero.com/ is down,hmmm down
8,1956969035,sadness,nic0lepaula,@charviray Charlene my love. I miss you,charlene love miss you
9,1956969172,sadness,Ingenue_Em,@kelcouch I'm sorry at least it's Friday?,sorry at least friday


#Lemmatization

In [58]:

from spacy.lang.en import English
nlp = spacy.load('en_core_web_sm')


In [61]:
def lemmatizer(text):
  tokens = [token.lemma_ for token in text]
  return ' '.join([token for token in tokens])


In [63]:
#data.clean_content = nlp(data.clean_content =)
data.clean_content = data.clean_content.apply(lambda x : lemmatizer(nlp(x)))


In [64]:
data.head(20)

,tweet_id,sentiment,author,content,clean_content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...,know listenin bad habit earlier and start frea...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...,layin n bed with headache ughhhh waitin on -PR...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...,funeral ceremony gloomy friday
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!,want hang out with friend soon
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...,want trade with someone houston ticket but no ...
5,1956968477,worry,xxxPEACHESxxx,Re-pinging @ghostridah14: why didn't you go to...,re ping why not -PRON- go prom bc bf not like ...
6,1956968487,sadness,ShansBee,"I should be sleep, but im not! thinking about ...",should sleep but not think about old friend wa...
7,1956968636,worry,mcsleazy,Hmmm. http://www.djhero.com/ is down,hmmm down
8,1956969035,sadness,nic0lepaula,@charviray Charlene my love. I miss you,charlene love miss -PRON-
9,1956969172,sadness,Ingenue_Em,@kelcouch I'm sorry at least it's Friday?,sorry at least friday


#Removing Empty Comments

In [65]:
data = data[data.clean_content != ""]

In [66]:
data.head(10)

,tweet_id,sentiment,author,content,clean_content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...,know listenin bad habit earlier and start frea...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...,layin n bed with headache ughhhh waitin on -PR...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...,funeral ceremony gloomy friday
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!,want hang out with friend soon
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...,want trade with someone houston ticket but no ...
5,1956968477,worry,xxxPEACHESxxx,Re-pinging @ghostridah14: why didn't you go to...,re ping why not -PRON- go prom bc bf not like ...
6,1956968487,sadness,ShansBee,"I should be sleep, but im not! thinking about ...",should sleep but not think about old friend wa...
7,1956968636,worry,mcsleazy,Hmmm. http://www.djhero.com/ is down,hmmm down
8,1956969035,sadness,nic0lepaula,@charviray Charlene my love. I miss you,charlene love miss -PRON-
9,1956969172,sadness,Ingenue_Em,@kelcouch I'm sorry at least it's Friday?,sorry at least friday


In [67]:
data.shape

(39894, 5)

In [68]:
data.sentiment.value_counts()

neutral       8563
worry         8453
happiness     5208
sadness       5161
love          3841
surprise      2183
fun           1776
relief        1526
hate          1323
empty          812
enthusiasm     759
boredom        179
anger          110
Name: sentiment, dtype: int64

In [69]:
sent_to_id  = {"empty":0, "sadness":1,"enthusiasm":2,"neutral":3,"worry":4,
                        "surprise":5,"love":6,"fun":7,"hate":8,"happiness":9,"boredom":10,"relief":11,"anger":12}

In [70]:
data["sentiment_id"] = data['sentiment'].map(sent_to_id)

In [71]:
data.head(10)

,tweet_id,sentiment,author,content,clean_content,sentiment_id
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...,know listenin bad habit earlier and start frea...,0
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...,layin n bed with headache ughhhh waitin on -PR...,1
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...,funeral ceremony gloomy friday,1
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!,want hang out with friend soon,2
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...,want trade with someone houston ticket but no ...,3
5,1956968477,worry,xxxPEACHESxxx,Re-pinging @ghostridah14: why didn't you go to...,re ping why not -PRON- go prom bc bf not like ...,4
6,1956968487,sadness,ShansBee,"I should be sleep, but im not! thinking about ...",should sleep but not think about old friend wa...,1
7,1956968636,worry,mcsleazy,Hmmm. http://www.djhero.com/ is down,hmmm down,4
8,1956969035,sadness,nic0lepaula,@charviray Charlene my love. I miss you,charlene love miss -PRON-,1
9,1956969172,sadness,Ingenue_Em,@kelcouch I'm sorry at least it's Friday?,sorry at least friday,1


In [72]:
data.to_csv('final_clean_text_emotions.csv')
